In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests10 as vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:
def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 200
trials =1000

skip_shi = True
refinement_test=True
adapt_c = False

data_tuned_c = .9
calc_c =lambda nobs: (nobs**(-1/2)/100,nobs**(1/5) )


In [5]:
# Input arrays
alpha = [0.01, 0.05, 0.10, 0.15]
results = np.array([
    [[0.98, 0.02, 0.00], [0.92, 0.08, 0.00]],
    [[0.86, 0.14, 0.00], [0.66, 0.34, 0.00]],
    [[0.66, 0.34, 0.00], [0.50, 0.50, 0.00]],
    [[0.54, 0.46, 0.00], [0.41, 0.59, 0.00]]
])
# Column headers

def print_mc2(alpha_levels ,test_results ):
    table = "\\begin{tabular}{c|cccc}\n"
    table += "\\hline\n"
    table += "\\textbf{$\\alpha$} & {} & \\textbf{No selection} & \\textbf{Model 1} & \\textbf{Model 2} \\\\\n"
    table += "\\hline\n"

    for i in range(len(alpha_levels)):
        alpha = alpha_levels[i]
        table += "{$%.2f$} & \\textbf{Normal} & %.2f & %.2f & %.2f \\\\\n" % (alpha, test_results[i][0][0], test_results[i][0][1], test_results[i][0][2])
        table += "& \\textbf{Bootstrap-ND} & %.2f & %.2f & %.2f \\\\\n" % (test_results[i][1][0], test_results[i][1][1], test_results[i][1][2])
        table += "\\hline\n"

    table += "\\end{tabular}"

    print(table)

#print_mc2(alpha,results)

# evidence of power

In [6]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9

def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [7]:
import vuong_tests5

setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)

res0 = vuong_tests5.monte_carlo(1,gen_data_ex,setup_shi_ex,trials=500,biascorrect=False)

nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
print(c1)

cstar 64
64 cstar 24.429530253249172
64 cstar 24.429530253249172
64 cstar 24.429530253249172
0.0006324555320336759


# a  = .25, k= 4,n=500

In [8]:
num_params=4
nobs=500
a1,a2 = np.sqrt(1.09**.5-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.63, 5th: 1.25, 10th: 1.73, 30th: 3.16, 50th: 4.58, 70th: 6.41, 90th: 9.90, 95th: 11.69, 99th: 16.75
bonus term 0.059139922087184965 // variance stats 0.2549022107716443 //part1 3.465724215775732 //part2 0.017064232006108338
-2.3044007602187757 2.734907835762893 2.6400265876341007
-2.4226806043931455 2.734907835762893 2.521746743459731
---
Quantiles: 1th: 0.58, 5th: 1.16, 10th: 1.64, 30th: 3.03, 50th: 4.38, 70th: 5.98, 90th: 9.05, 95th: 11.24, 99th: 15.09
bonus term 0.06900719024478069 // variance stats 0.1991753321175424 //part1 3.465724215775732 //part2 0.019911333374613257
-2.280196037405014 2.0823905563609504 2.562082528374029
-2.4182104178945756 2.0823905563609504 2.4240681478844675
---
Quantiles: 1th: 0.62, 5th: 1.20, 10th: 1.71, 30th: 3.17, 50th: 4.58, 70th: 6.46, 90th: 9.72, 95th: 11.70, 99th: 16.34
bonus term 0.07947580715990148 // variance stats 0.18522333085107648 //part1 3.465724215775732 //part2 0.022931947902297942
-2.5176644546074405 1.6104695267589921 2

Quantiles: 1th: 0.57, 5th: 1.10, 10th: 1.54, 30th: 2.69, 50th: 3.88, 70th: 5.31, 90th: 8.12, 95th: 9.86, 99th: 13.86
bonus term 0.043759240872774795 // variance stats 0.2838424823018995 //part1 3.465724215775732 //part2 0.012626290537944656
-1.5234892224710743 1.361655380915907 1.5302335094818766
-1.6110077042166238 1.361655380915907 1.4427150277363272
---
Quantiles: 1th: 0.62, 5th: 1.33, 10th: 1.76, 30th: 3.27, 50th: 4.70, 70th: 6.53, 90th: 10.11, 95th: 12.28, 99th: 17.96
bonus term 0.06250599464902386 // variance stats 0.24628900489010988 //part1 3.465724215775732 //part2 0.0180354785197567
-1.2502542701418164 2.0212402902911544 1.592596252243561
-1.3752662594398641 2.0212402902911544 1.4675842629455134
---
Quantiles: 1th: 0.65, 5th: 1.27, 10th: 1.75, 30th: 3.13, 50th: 4.47, 70th: 6.23, 90th: 9.82, 95th: 11.76, 99th: 16.12
bonus term 0.05186083125879474 // variance stats 0.28922946765806695 //part1 3.465724215775732 //part2 0.01496392327546661
-1.2976123423972397 3.119524182567129 1.

# a  = .25, k= 4, n=250

In [9]:
num_params=4
nobs=250
a1,a2 = np.sqrt(1.09-1), 0.00
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data2(nobs=nobs, a1=a1, a2=a2, num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([reg,boot3])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.70, 5th: 1.23, 10th: 1.67, 30th: 3.00, 50th: 4.33, 70th: 6.04, 90th: 9.24, 95th: 11.13, 99th: 15.87
bonus term 0.05505902128267127 // variance stats 0.3144209529374689 //part1 3.017088168272582 //part2 0.01824905942811576
-2.5055375661567796 2.418985186549696 2.6258382806391185
-2.615655608722122 2.418985186549696 2.515720238073776
---
Quantiles: 1th: 0.67, 5th: 1.23, 10th: 1.73, 30th: 3.00, 50th: 4.36, 70th: 6.13, 90th: 9.43, 95th: 11.33, 99th: 15.39
bonus term 0.05688920385141392 // variance stats 0.31049395791710216 //part1 3.017088168272582 //part2 0.018855665024859226
-2.206410897718358 2.104219576385158 2.432376414137878
-2.3201893054211857 2.104219576385158 2.31859800643505
---
Quantiles: 1th: 0.78, 5th: 1.52, 10th: 2.03, 30th: 3.56, 50th: 5.09, 70th: 7.07, 90th: 10.81, 95th: 13.05, 99th: 17.78
bonus term 0.05658187587897502 // variance stats 0.35780812215954394 //part1 3.017088168272582 //part2 0.018753802581570125
-2.1652256984565565 2.130349798872352 2.71601

Quantiles: 1th: 0.79, 5th: 1.38, 10th: 1.86, 30th: 3.42, 50th: 5.01, 70th: 7.05, 90th: 10.70, 95th: 12.81, 99th: 17.59
bonus term 0.07741406133684911 // variance stats 0.2570304020207085 //part1 3.017088168272582 //part2 0.025658534659652364
-1.5678190050836256 -0.474441531733257 1.7879082504442363
-1.7226471277573236 -0.474441531733257 1.6330801277705382
---
Quantiles: 1th: 0.65, 5th: 1.12, 10th: 1.57, 30th: 2.81, 50th: 4.05, 70th: 5.56, 90th: 8.51, 95th: 10.39, 99th: 15.01
bonus term 0.05243538076300875 // variance stats 0.3042462208362122 //part1 3.017088168272582 //part2 0.01737946584207062
-1.3763736877488564 2.672907901449987 1.6973222352617245
-1.4812444492748738 2.672907901449987 1.5924514737357072
---
Quantiles: 1th: 0.93, 5th: 1.60, 10th: 2.14, 30th: 3.70, 50th: 5.33, 70th: 7.21, 90th: 10.97, 95th: 13.43, 99th: 18.62
bonus term 0.045930320447355816 // variance stats 0.4486208041088144 //part1 3.017088168272582 //part2 0.01522339351244514
-1.3868598368181153 3.5555491118141442

# size stuff

# a  = .25, k= 4,n=250

In [10]:
num_params=4
nobs=250
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,
                                          data_tuned_c=data_tuned_c,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.66, 5th: 1.22, 10th: 1.61, 30th: 2.94, 50th: 4.27, 70th: 5.93, 90th: 9.01, 95th: 11.17, 99th: 15.34
bonus term 0.06249037223776051 // variance stats 0.26949703384734275 //part1 3.017088168272582 //part2 0.020712146530851648
-2.4937846365740763 0.36481390059761504 2.5441253877778682
-2.6187653810495974 0.36481390059761504 2.419144643302347
---
Quantiles: 1th: 0.74, 5th: 1.34, 10th: 1.82, 30th: 3.34, 50th: 4.69, 70th: 6.52, 90th: 9.95, 95th: 11.94, 99th: 16.37
bonus term 0.05514264633594051 // variance stats 0.3380685980467654 //part1 3.017088168272582 //part2 0.01827677656749161
-2.259802709846764 -0.7947238173700856 2.327986910278479
-2.370088002518645 -0.7947238173700856 2.2177016176065982
---
Quantiles: 1th: 0.72, 5th: 1.30, 10th: 1.74, 30th: 3.06, 50th: 4.44, 70th: 6.13, 90th: 9.55, 95th: 11.62, 99th: 16.58
bonus term 0.054020659987482 // variance stats 0.3313017638449234 //part1 3.017088168272582 //part2 0.017904899351486717
-2.582758702696472 2.093239332268943 2.

Quantiles: 1th: 0.68, 5th: 1.35, 10th: 1.80, 30th: 3.18, 50th: 4.60, 70th: 6.39, 90th: 9.59, 95th: 11.83, 99th: 16.45
bonus term 0.047553196054005535 // variance stats 0.37887478612883985 //part1 3.017088168272582 //part2 0.015761288169855465
-1.6034351875442847 -0.3890212147803095 1.6951080433287191
-1.6985415796522956 -0.3890212147803095 1.6000016512207083
---
Quantiles: 1th: 0.70, 5th: 1.30, 10th: 1.73, 30th: 3.09, 50th: 4.39, 70th: 6.09, 90th: 9.24, 95th: 11.15, 99th: 15.73
bonus term 0.044959974804515204 // variance stats 0.38614272178118997 //part1 3.017088168272582 //part2 0.014901776911033
-1.3523699458264244 0.29996735299556637 1.5632419874247778
-1.442289895435455 0.29996735299556637 1.4733220378157472
---
Quantiles: 1th: 0.78, 5th: 1.51, 10th: 2.06, 30th: 3.63, 50th: 5.13, 70th: 7.16, 90th: 10.95, 95th: 13.35, 99th: 18.30
bonus term 0.048232464581337314 // variance stats 0.42636337503938443 //part1 3.017088168272582 //part2 0.015986428599782207
-1.3136613026026898 0.08977319

# a  = .25, k= 4,n=500

In [11]:
###### num_params=4
nobs=500
a=.25 
c1,c2 = calc_c(nobs)
alphas = [.01,.05,.1,.15]
test_results = []

for alpha in alphas:
    setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
    gen_data_ex = lambda : gen_data(nobs=nobs, a=a,  num_params=num_params)
    mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi_ex,trials=trials,c1=c1,c2=c2,adapt_c=adapt_c,
                                          skip_shi=skip_shi, refinement_test=refinement_test,alpha=alpha)
    reg,twostep, refine_test, boot1,boot2,boot3,shi, llr,std, omega = mc_out
    test_results.append([refine_test,boot3])
print_mc2(alphas,np.array(test_results))

Quantiles: 1th: 0.75, 5th: 1.34, 10th: 1.84, 30th: 3.35, 50th: 4.78, 70th: 6.67, 90th: 10.26, 95th: 12.53, 99th: 17.93
bonus term 0.0044766643823281336 // variance stats 0.34577462819963733 //part1 3.465724215775732 //part2 0.0012916966566325946
-2.6262681542313353 0.3746880075513154 2.423663712038647
-2.6352214829959917 0.3746880075513154 2.4147103832739907
---
Quantiles: 1th: 0.82, 5th: 1.48, 10th: 2.05, 30th: 3.56, 50th: 5.01, 70th: 6.89, 90th: 10.51, 95th: 12.52, 99th: 17.44
bonus term 0.0034000998952287147 // variance stats 0.455397918991135 //part1 3.465724215775732 //part2 0.0009810647597843186
-2.400045606545154 1.0375569965727611 2.3762765767909126
-2.4068458063356113 1.0375569965727611 2.3694763770004554
---
Quantiles: 1th: 0.78, 5th: 1.38, 10th: 1.92, 30th: 3.44, 50th: 4.90, 70th: 6.82, 90th: 10.34, 95th: 12.47, 99th: 17.83
bonus term 0.003690378834293709 // variance stats 0.41954191411826003 //part1 3.465724215775732 //part2 0.0010648218393995012
-2.4419214191842658 -0.5902

-1.6574625580559688 0.9086146732325643 1.6221754215057975
-1.666404560201196 0.9086146732325643 1.6132334193605702
---
Quantiles: 1th: 0.73, 5th: 1.44, 10th: 1.89, 30th: 3.36, 50th: 4.82, 70th: 6.77, 90th: 10.06, 95th: 12.19, 99th: 16.59
bonus term 0.0037904654663538906 // variance stats 0.40845216863241124 //part1 3.465724215775732 //part2 0.00109370083433066
-1.672029376487844 -0.36028385260738593 1.649917814159951
-1.6796103074205517 -0.36028385260738593 1.6423368832272434
---
Quantiles: 1th: 0.69, 5th: 1.22, 10th: 1.65, 30th: 2.86, 50th: 4.01, 70th: 5.55, 90th: 8.36, 95th: 9.94, 99th: 13.06
bonus term 0.004285791096764335 // variance stats 0.3611940686215655 //part1 3.465724215775732 //part2 0.0012366220824079758
-1.3705179731615478 0.4701269351044404 1.4316350916518663
-1.3790895553550766 0.4701269351044404 1.4230635094583375
---
Quantiles: 1th: 0.74, 5th: 1.32, 10th: 1.76, 30th: 3.14, 50th: 4.43, 70th: 6.10, 90th: 9.37, 95th: 11.43, 99th: 15.26
bonus term 0.004782945216973684 // 